In [100]:
# Read in the data
CPCG0100Frame = read.csv("CPCG0100.csv")

In [101]:
str(CPCG0100Frame)

'data.frame':	198526 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  183 177 115 83 96 120 117 92 27 9 ...
 $ NonRefAllele   : int  3 10 10 26 20 11 14 8 3 3 ...
 $ BaseQual       : int  4490 4691 3512 3251 3255 3672 3865 2946 524 342 ...
 $ TumourCoverage : int  231 229 137 119 127 155 155 123 36 17 ...
 $ NormalCoverage : int  133 134 74 59 53 108 108 75 26 15 ...
 $ MapQual        : num  25 25 0 37 29 0 0 0 27 0 ...
 $ ReadPosition   : num  55 60 52 38 47 42 51 55 70.5 64 ...
 $ Trinucleotide  : Factor w/ 35 levels "ACA","ACC","ACG",..: 32 14 28 18 19 24 34 18 35 6 ...
 $ HomopolymerRate: num  4.7 4.53 2.71 2.75 2.72 ...
 $ GCcontent      : num  0.587 0.587 0.532 0.677 0.677 ...
 $ GenomicLocation: Factor w/ 16 levels "","downstream",..: 1 1 1 8 1 1 8 1 1 5 ...
 $ DistanceSNP    : int  41830 41825 35658 34719 34700 32011 31988 23727 23645 10933 ...
 $ StrandBias     : num  0.368 0.371 0.336 0.462 0.409 ...
 $ Label          : num  0 0

In [102]:
sum(is.na(CPCG0100Frame$StrandBias)) 

[1] 32

In [103]:
sum(is.na(CPCG0100Frame$ReadPosition)) 

[1] 32

In [104]:
CPCG0100Frame$ReadPosition[is.na(CPCG0100Frame$ReadPosition)] = mean(CPCG0100Frame$ReadPosition, na.rm=TRUE)
CPCG0100Frame$StrandBias[is.na(CPCG0100Frame$StrandBias)] = mean(CPCG0100Frame$StrandBias, na.rm=TRUE)

print (sum(is.na(CPCG0100Frame$ReadPosition)) )
print (sum(is.na(CPCG0100Frame$StrandBias)) )

[1] 0
[1] 0


In [105]:
sum(is.na(CPCG0100Frame))  #No NA value now

[1] 0

In [106]:
write.csv(CPCG0100Frame, file = "CPCG0100_imputed.csv")  #Cleaned CSV without NA

In [10]:
CPCG0100Frame$Trinucleotide  = as.factor(CPCG0100Frame$Trinucleotide)
CPCG0100Frame$GenomicLocation= as.factor(CPCG0100Frame$GenomicLocation)

In [11]:
library(caTools)
split = sample.split(CPCG0100Frame$Y, SplitRatio = 0.80) #60% train, 40% test
train_R = subset(CPCG0100Frame, split==TRUE)
test_R = subset(CPCG0100Frame, split==FALSE)

In [12]:
CPCG0100Log = glm(Y ~ ., data = CPCG0100Frame, family=binomial)
summary(CPCG0100Log)

Warning message:
: glm.fit: algorithm did not convergeWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred


Call:
glm(formula = Y ~ ., family = binomial, data = CPCG0100Frame)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-8.4904  -0.1760  -0.0841  -0.0134   8.4904  

Coefficients:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                        -7.527e+00  3.224e-01 -23.343  < 2e-16 ***
X                                  -2.117e-06  3.377e-07  -6.269 3.62e-10 ***
RefAllele                          -1.213e-01  5.536e-03 -21.917  < 2e-16 ***
NonRefAllele                       -6.052e-02  5.290e-03 -11.441  < 2e-16 ***
BaseQual                            4.522e-03  1.692e-04  26.727  < 2e-16 ***
TumourCoverage                     -3.752e-02  2.775e-03 -13.519  < 2e-16 ***
NormalCoverage                     -2.038e-02  1.503e-03 -13.558  < 2e-16 ***
MapQual                             1.284e-01  4.074e-03  31.508  < 2e-16 ***
ReadPosition                        3.865e-03  2.066e-03   1.871 0.061351 .  
TrinucleotideACC      

In [13]:
predictCPCG0100 = predict(CPCG0100Log, type="response")

In [14]:
table(CPCG0100Frame$Y, predictCPCG0100 > 0.5)

   
     FALSE   TRUE
  0 195527    146
  1   2835     18

In [15]:
(195527+18)/(195527+18+146+2835.0)

[1] 0.9849843

In [16]:
#################################################################
############# H2o Deep Learning

In [107]:
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpYwTNS9/h2o_rbhat_started_from_r.out
    /tmp/RtmpYwTNS9/h2o_rbhat_started_from_r.err


Starting H2O JVM and connecting: .. Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 seconds 669 milliseconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_odx059 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.60 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 

Note:  As started, H2O is limited to the CRAN default of 2 CPUs.
       Shut down and restart H2O as shown below to use all your CPUs.
           > h2o.s

In [111]:
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/CPCG0100_imputed.csv"
CPCG0100.hex          = h2o.importFile(path = csvPath, destination_frame = "CPCG0100.hex") 
train                 = as.data.frame(CPCG0100.hex)

#train  = subset(train,C1<20000)

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%
  |======================================================================| 100%


In [116]:
str(train_h2o)

Class 'H2OFrame' <environment: 0x7390a50> 
 - attr(*, "op")= chr "Parse"
 - attr(*, "id")= chr "train"
 - attr(*, "eval")= logi FALSE
 - attr(*, "nrow")= int 198526
 - attr(*, "ncol")= int 16
 - attr(*, "types")=List of 16
  ..$ : chr "int"
  ..$ : chr "int"
  ..$ : chr "int"
  ..$ : chr "int"
  ..$ : chr "int"
  ..$ : chr "int"
  ..$ : chr "int"
  ..$ : chr "real"
  ..$ : chr "real"
  ..$ : chr "enum"
  ..$ : chr "real"
  ..$ : chr "real"
  ..$ : chr "enum"
  ..$ : chr "int"
  ..$ : chr "real"
  ..$ : chr "enum"
 - attr(*, "data")='data.frame':	10 obs. of  16 variables:
  ..$ C1             : num  1 2 3 4 5 6 7 8 9 10
  ..$ X              : num  0 1 2 3 4 5 6 7 8 9
  ..$ RefAllele      : num  183 177 115 83 96 120 117 92 27 9
  ..$ NonRefAllele   : num  3 10 10 26 20 11 14 8 3 3
  ..$ BaseQual       : num  4490 4691 3512 3251 3255 ...
  ..$ TumourCoverage : num  231 229 137 119 127 155 155 123 36 17
  ..$ NormalCoverage : num  133 134 74 59 53 108 108 75 26 15
  ..$ MapQual        : n

In [113]:
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [141]:
#var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1,1,0.332800319279481
2,TumourCoverage,0.146610274910927,0.146610274910927,0.0487919463000089
3,NormalCoverage,0.133089661598206,0.133089661598206,0.0442922818726809
4,BaseQual,0.108282387256622,0.108282387256622,0.0360364130513483
5,RefAllele,0.0907380729913712,0.0907380729913712,0.0301976596623332
6,GenomicLocation.upstream,0.075579859316349,0.075579859316349,0.0251530013115792
7,Trinucleotide.GTG,0.0612118318676949,0.0612118318676949,0.0203713171892508
8,GenomicLocation.ncRNA_UTR5,0.0594158470630646,0.0594158470630646,0.0197736128728487
9,MapQual,0.0566472858190536,0.0566472858190536,0.0188522348068971
10,GenomicLocation.ncRNA_splicing,0.0556206367909908,0.0556206367909908,0.0185105656825698


In [142]:

write.table(var_importance, file="CPCG0100_variable_importance.csv")

In [119]:
#Predict on test data:
#test_h2o <- h2o.importFile(path = '/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/test.csv',
#                           destination_frame = "test_h2o")

yhat <- h2o.predict(model, train_h2o)
LabelId <- as.numeric(seq(1,198526))
names(LabelId)[1] <- "LabelId"
predictions <- cbind(as.data.frame(LabelId),as.data.frame(yhat[,1]))
names(predictions)[2] <- "Label"
#write.table(as.matrix(predictions), file="DNN_pred.csv", row.names=FALSE, sep=",")

  |======================================================================| 100%


In [120]:
str(predictions)

'data.frame':	198526 obs. of  2 variables:
 $ LabelId: num  1 2 3 4 5 6 7 8 9 10 ...
 $ Label  : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 1 1 ...


In [121]:
sum(train$Label==predictions$Label)

[1] 172969

In [122]:
172969/198526.

[1] 0.8712662

In [53]:
train_h2o[c(3),c(2,15)]

  RefAllele Y
1       115 0

[1 row x 2 columns] 

In [49]:
subset(CPCG0100Frame, X < 3)

,X,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,DistanceSNP,StrandBias,Y
1,0,183,3,4490,231,133,25,55,TTA,4.696078,0.5870647,,41830,0.3679654,0
2,1,177,10,4691,229,134,25,60,CTA,4.528846,0.5870647,,41825,0.371179,0
3,2,115,10,3512,137,74,0,52,TCA,2.707143,0.5323383,,35658,0.3357664,0


In [91]:
str(subset(train,C1<20000))

'data.frame':	19999 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  183 177 115 83 96 120 117 92 27 9 ...
 $ NonRefAllele   : int  3 10 10 26 20 11 14 8 3 3 ...
 $ BaseQual       : int  4490 4691 3512 3251 3255 3672 3865 2946 524 342 ...
 $ TumourCoverage : int  231 229 137 119 127 155 155 123 36 17 ...
 $ NormalCoverage : int  133 134 74 59 53 108 108 75 26 15 ...
 $ MapQual        : num  25 25 0 37 29 0 0 0 27 0 ...
 $ ReadPosition   : num  55 60 52 38 47 42 51 55 70.5 64 ...
 $ Trinucleotide  : Factor w/ 35 levels "ACA","ACC","ACG",..: 32 14 28 18 19 24 34 18 35 6 ...
 $ HomopolymerRate: num  4.7 4.53 2.71 2.75 2.72 ...
 $ GCcontent      : num  0.587 0.587 0.532 0.677 0.677 ...
 $ GenomicLocation: Factor w/ 15 levels "downstream","exonic",..: NA NA NA 7 NA NA 7 NA NA 4 ...
 $ DistanceSNP    : int  41830 41825 35658 34719 34700 32011 31988 23727 23645 10933 ...
 $ StrandBias     : num  

In [32]:
n = c(2, 3, 5) 
s = c("aa", "bb", "cc") 
b = c(TRUE, FALSE, TRUE) 
df = data.frame(n, s, b)       # df is a data frame

In [33]:
df

,n,s,b
1,2,aa,TRUE
2,3,bb,FALSE
3,5,cc,TRUE


In [41]:
df[2,c(2,3)]

,s,b
2,bb,FALSE


In [47]:
df[c(1,3),]

,n,s,b
1,2,aa,TRUE
3,5,cc,TRUE


In [60]:
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")
## Import MNIST CSV as H2O:
mnistPath = '/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/train.csv'
mnist.hex = h2o.importFile(path = mnistPath, destination_frame = "mnist.hex")
train <- as.data.frame(mnist.hex)
train$label <- as.factor(train$label)
train_h2o <- as.h2o(train)

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         1 minutes 46 seconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_css426 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.60 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 

  |======================================================================| 100%
  |======================================================================| 100%


In [63]:
train[c(1,2,3,4),]

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,ellip.h,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [66]:
#str(train)

In [68]:

#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning(x = 2:785,
                   y = 1,
                   training_frame = train_h2o,
                   activation = "RectifierWithDropout", 
                   input_dropout_ratio = 0.2,
                   hidden_dropout_ratios = c(0.5,0.5),
                   balance_classes = TRUE, 
                   hidden = c(200,100), #c(800,800),
                   epochs = 10)


Warning message:
In .h2o.validateModelParameters(algo, param_values, h2oRestApiVersion): Dropping constant columns: [pixel673, pixel196, pixel672, pixel671, pixel57, pixel476, pixel10, pixel532, pixel11, pixel16, pixel52, pixel730, pixel53, pixel18, pixel731, pixel54, pixel17, pixel392, pixel55, pixel56, pixel19, pixel782, pixel2, pixel781, pixel3, pixel780, pixel4, pixel5, pixel448, pixel0, pixel783, pixel168, pixel1, pixel700, pixel701, pixel645, pixel644, pixel112, pixel111, pixel7, pixel6, pixel9, pixel8, pixel759, pixel758, pixel757, pixel756, pixel755, pixel754, pixel31, pixel30, pixel699, pixel139, pixel729, pixel141, pixel140, pixel727, pixel728, pixel23, pixel22, pixel21, pixel20, pixel560, pixel29, pixel28, pixel82, pixel27, pixel83, pixel26, pixel421, pixel84, pixel25, pixel760, pixel420, pixel85, pixel24].


  |======================================================================| 100%


In [69]:
#Predict on test data:
test_h2o <- h2o.importFile(path = '/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/test.csv',
                           destination_frame = "test_h2o")

yhat <- h2o.predict(model, test_h2o)
ImageId <- as.numeric(seq(1,28000))
names(ImageId)[1] <- "ImageId"
predictions <- cbind(as.data.frame(ImageId),as.data.frame(yhat[,1]))
names(predictions)[2] <- "Label"
#write.table(as.matrix(predictions), file="DNN_pred.csv", row.names=FALSE, sep=",")

  |======================================================================| 100%
  |======================================================================| 100%


In [71]:
str(predictions)

'data.frame':	28000 obs. of  2 variables:
 $ ImageId: num  1 2 3 4 5 6 7 8 9 10 ...
 $ Label  : Factor w/ 10 levels "0","1","2","3",..: 3 1 10 10 4 8 1 4 1 4 ...
